In [18]:
from i2 import Sig
from typing import Any
from front.scrap.py2pydantic import func_to_pyd_model_specs, func_to_pyd_input_model_cls, func_to_pyd_func

from i2.tests.objects_for_testing import formula1


list(func_to_pyd_model_specs(func=formula1, dflt_type=Any))

[('w', (typing.Any, Ellipsis)),
 ('x', (float, Ellipsis)),
 ('y', 1),
 ('z', (int, 1))]

In [19]:
func = formula1
pyd_model = func_to_pyd_input_model_cls(func, dflt_type=Any)
    

In [20]:
pyd_model

pydantic.main.formula1

In [21]:
pyd_model.schema()

{'title': 'formula1',
 'type': 'object',
 'properties': {'w': {'title': 'W'},
  'x': {'title': 'X', 'type': 'number'},
  'z': {'title': 'Z', 'default': 1, 'type': 'integer'},
  'y': {'title': 'Y', 'default': 1, 'type': 'integer'}},
 'required': ['w', 'x']}

In [22]:
def pyd_func_ingress_template(input_model_instance, wrapped_func_sig: Sig):
    kwargs = dict(input_model_instance)
    args, kwargs = wrapped_func_sig.args_and_kwargs_from_kwargs(kwargs)
    return args, kwargs

input_model_instance = pyd_model(w=1, x=2, y=3)

In [23]:
kwargs = dict(input_model_instance)
wrapped_func_sig = Sig(formula1)
args, kwargs = wrapped_func_sig.args_and_kwargs_from_kwargs(kwargs)
args, kwargs

((1,), {'x': 2.0, 'y': 3, 'z': 1})

In [24]:
def func_to_pyd_func(func: Callable, dflt_type=Any):
    """Get a 'opyrator' function from a python function.
    That is, a function that has a single pydantic model input and output.
    """
    pyd_func_ingress = partial(pyd_func_ingress_template, wrapped_func_sig=Sig(func))

    input_model = func_to_pyd_input_model_cls(func, dflt_type)
    output_model = create_model(
        "output_model", output_val=(Any, ...)
    )  # TODO: Work on this
    # TODO: Inject annotations in pyd_func_ingress

    return wrap(func, ingress=pyd_func_ingress)

NameError: name 'Callable' is not defined

In [25]:
from typing import Any
pyd_func = func_to_pyd_func(formula1, dflt_type=Any)

In [26]:
input_model = func_to_pyd_input_model_cls(formula1, dflt_type=Any)

In [28]:
inp = input_model(w=2,x=1,y=2)
inp

formula1(w=2, x=1.0, z=1, y=2)

In [29]:
pyd_func(inp)

6.0

In [33]:
def double_string(x:str)->int:
    return len(2*x)

sig = Sig(double_string)

In [42]:
from pydantic import create_model
from typing import Callable, Any

def func_to_pyd_output_model_cls(func: Callable, dflt_type=Any):
    return_type = Sig(func).return_annotation

    output = create_model('Output', output_val = return_type)

    return output

In [44]:
output = func_to_pyd_output_model_cls(formula1)

In [48]:
res = output(output_val =3)

In [49]:
res.__fields__

{}

In [50]:
def egress(output):
    return_type = type(output)
    mod = create_model('Output', output_val = return_type)
    
    return mod(output_val = output)